In [2]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

import croniter
import pandas as pd
import datetime
from datetime import timedelta
import re

# Croniter

In [529]:
now = datetime.datetime.now()
iter = croniter.croniter('55 23 * * Sat,Mon-Fri 55', pd.Timestamp('2023-08-25'))
iter = croniter.croniter('55 23 * * Sat 55', pd.Timestamp('2023-08-25'))
iter = croniter.croniter('* * * * * */55', pd.Timestamp('2023-08-25 12:00'))

print(pd.Timestamp(iter.get_prev(), unit='s'))
print('- - - -')
for _ in range(10):
    print(pd.Timestamp(iter.get_next(), unit='s'))

2023-08-25 11:59:55
- - - -
2023-08-25 12:00:00
2023-08-25 12:00:55
2023-08-25 12:01:00
2023-08-25 12:01:55
2023-08-25 12:02:00
2023-08-25 12:02:55
2023-08-25 12:03:00
2023-08-25 12:03:55
2023-08-25 12:04:00
2023-08-25 12:04:55


In [6]:
# # cmp = pd.Timedelta("6d23h55min").components
# cmp = pd.Timedelta("5Min").components
# sched = f"{cmp.minutes} {cmp.hours} */{cmp.days} * * {cmp.seconds}"
# iter = croniter.croniter(sched, pd.Timestamp('2023-08-25 12:00'))

# pd.Timestamp(iter.get_prev(), unit='s'), pd.Timestamp(iter.get_next(), unit='s')

In [ ]:
from dateutil.parser import parser
from dateutil.rrule import rrulestr
# list(rrulestr("""RRULE:FREQ=DAILY;INTERVAL=10"""))
list(rrulestr('FREQ=WEEKLY;INTERVAL=1;BYHOUR=23;BYMINUTE=45;BYSECOND=0;COUNT=5'))
list(rrulestr('FREQ=WEEKLY;INTERVAL=1;BYHOUR=23;BYMINUTE=45;BYSECOND=0;COUNT=5'))

# Spec parser

In [408]:
%load_ext autoreload
%autoreload 2
from qubx.core.helpers import BasicScheduler, process_schedule_spec
from qubx.utils.time import convert_seconds_to_str

In [411]:
specs = [
    '',
    'bar.1m: -5Sec',        # 5 sec before subsciption bar closing
    '1Min -5Sec',           # 5 sec before subsciption bar closing
    'bar:  5Sec',           # 5 sec after subsciption bar opened
    'bar:  +5Sec',          # 5 sec after subsciption bar opened
    'time: 23:59:00 @ Sat', # at 23:59:00 every Saturday
    'time: 23:59 @ Sat',    # at 23:59:00 every Saturday
    'time: 9:30:10',        # every day at 9:30:10
    '* * * * * 45',         # raw cron 
    'cron: * * * * * 45',   # raw cron witch implicit type
    '23:59:00 @ Sun',       # at 23:59:00 every Sunday
    '9:30:10',              # every day at 9:30:10
    '9:30 @ Mon-Fri',       # mon, ... fri at 9:30
    '9:30:22 @ Mon,Fri',    # mon and fri at 9:30:22
    '9:30 , 15:45 @ Mon-Fri', # mon, ... fri at 9:30 and 15:45
    '9:30, 15:45 @ Mon, Fri', # mon and fri at 9:30 and 15:45
    '9:30:10  15:45 @ Mon, Fri', # mon and fri at 9:30 and 15:45
    'calendar: 23:00 liquidation', # custom event
    '9:30 -1Min'            # time 
    '-5Sec',                # 5 sec before subsciption bar end
    '5Min -5Sec',           # 5 sec before every 5min interval ends
    '1w -1h',               # 1h before every week ends
    '4month -5hour',        # 5h before the end of every 4 months 
    '1week -3hours -10Mins',# 5h before the end of every 4 months 
    '2h30m -5Sec',          # 5 sec before every 2h and 30min interval ends
    '1w @ Sat',             # 5 sec before every 2h and 30min interval ends
]

In [415]:
for s in specs:
    print(s.ljust(30), process_schedule_spec(s))

                               []
bar.1m: -5Sec                  [{'type': 'bar', 'args': None, 'timeframe': '1m', 'delay': Timedelta('-1 days +23:59:55'), 'spec': '-5Sec'}]
1Min -5Sec                     [{'type': 'bar', 'args': None, 'timeframe': '1Min', 'delay': Timedelta('-1 days +23:59:55'), 'spec': '1Min -5Sec'}]
bar:  5Sec                     [{'type': 'bar', 'args': None, 'timeframe': None, 'delay': Timedelta('0 days 00:00:05'), 'spec': '5Sec'}]
bar:  +5Sec                    [{'type': 'bar', 'args': None, 'timeframe': None, 'delay': Timedelta('0 days 00:00:05'), 'spec': '+5Sec'}]
time: 23:59:00 @ Sat           [{'type': 'cron', 'args': '59 23 * * Sat', 'spec': '23:59:00 @ Sat'}]
time: 23:59 @ Sat              [{'type': 'cron', 'args': '59 23 * * Sat', 'spec': '23:59 @ Sat'}]
time: 9:30:10                  [{'type': 'cron', 'args': '30 9 * * * 10', 'spec': '9:30:10'}]
* * * * * 45                   [{'type': 'cron', 'args': '* * * * * 45', 'spec': '* * * * * 45'}]
cron: * * * *

In [414]:
if 0:
    for s in specs: 
        m = re.compile(
            r"((?P<type>[A-Za-z]+)(\.?(?P<timeframe>[0-9A-Za-z]+))?\ *:)?"
            r"\ *"
            r"((?P<spec>"
                r"(?P<time>((\d+:\d+(:\d+)?)\ *,?\ *)+)?"
                r"((\ *@\ *)(?P<by>([A-Za-z0-9-,\ ]+)))?"
                r"(("
                r'((?P<months>[-+]?\d+)(months|month|bm|mo))?'
                r'((?P<weeks>[-+]?\d+)(weeks|week|w))?'
                r'((?P<days>[-+]?\d+)(days|day|d))?'
                r'((?P<hours>[-+]?\d+)(hours|hour|h))?'
                r'((?P<minutes>[-+]?\d+)(mins|min|m))?'
                r'((?P<seconds>[-+]?\d+)(sec|s))?'
                r")(\ *)?)*"
                r".*"
            r"))?"

            , re.IGNORECASE
        ).match(s)
        print(s.ljust(30), '->', {k: v for k, v in m.groupdict().items() if v} if m else '?????')


# Scheduler tests

In [454]:
%load_ext autoreload
%autoreload 2
from qubx.core.helpers import BasicScheduler
from qubx.core.basics import CtrlChannel
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
if 0:
    import sched
    import time as time_module

    def myfunc(): 
        print(f"Trigger: {pd.Timestamp('now')}")

    scheduler = sched.scheduler(time_module.time, time_module.sleep)
    t1 = (pd.Timestamp('now', tz='UTC') + pd.Timedelta('3s')).round('3s').timestamp()
    t2 = (pd.Timestamp('now', tz='UTC') + pd.Timedelta('5s')).round('5s').timestamp()
    scheduler_e = scheduler.enterabs(t1, 1, myfunc, ())
    scheduler_e = scheduler.enterabs(t2, 1, myfunc, ())

    while (r:=scheduler.run(blocking=False)):
        time_module.sleep(max(r/5, 0.1))

In [415]:
time_now = lambda: pd.Timestamp('now', tz='UTC').as_unit('ns').asm8.item()

bs = BasicScheduler(c:=CtrlChannel('test'), time_now)
bs.schedule_event('* * * * * */10', 'TEST')

In [416]:
bs.get_event_last_time('TEST'), bs.get_event_next_time('TEST')

(Timestamp('2024-05-04 11:09:50'), Timestamp('2024-05-04 11:10:00'))

In [417]:
bs.run()

2024-05-04 14:09:59.699 [ 🐞 ] Next (TEST) event scheduled at 2024-05-04 11:10:00


2024-05-04 14:10:00.002 [ 🐞 ] Next (TEST) event scheduled at 2024-05-04 11:10:10
2024-05-04 14:10:10.067 [ 🐞 ] Next (TEST) event scheduled at 2024-05-04 11:10:20
2024-05-04 14:10:15.017 [ 🐞 ] Next (TEST-xxxx) event scheduled at 2024-05-04 11:10:16
2024-05-04 14:10:16.001 [ 🐞 ] Next (TEST-xxxx) event scheduled at 2024-05-04 11:10:17
2024-05-04 14:10:17.097 [ 🐞 ] Next (TEST-xxxx) event scheduled at 2024-05-04 11:10:18
2024-05-04 14:10:18.043 [ 🐞 ] Next (TEST-xxxx) event scheduled at 2024-05-04 11:10:19
2024-05-04 14:10:19.013 [ 🐞 ] Next (TEST-xxxx) event scheduled at 2024-05-04 11:10:20
2024-05-04 14:10:20.000 [ 🐞 ] Next (TEST) event scheduled at 2024-05-04 11:10:30
2024-05-04 14:10:20.000 [ 🐞 ] Next (TEST-xxxx) event scheduled at 2024-05-04 11:10:21
2024-05-04 14:10:21.095 [ 🐞 ] Next (TEST-xxxx) event scheduled at 2024-05-04 11:10:22
2024-05-04 14:10:22.042 [ 🐞 ] Next (TEST-xxxx) event scheduled at 2024-05-04 11:10:23
2024-05-04 14:10:23.013 [ 🐞 ] Next (TEST-xxxx) event scheduled at 202

In [420]:
bs.run()

2024-05-04 14:14:40.920 [ 🐞 ] Next (TEST) event scheduled at 2024-05-04 11:14:50
2024-05-04 14:14:40.926 [ 🐞 ] Next (TEST-xxxx) event scheduled at 2024-05-04 11:14:41
2024-05-04 14:14:40.926 [ 🐞 ] Next (TEST-xxxx) event scheduled at 2024-05-04 11:14:41
2024-05-04 14:14:40.926 [ 🐞 ] Next (TEST) event scheduled at 2024-05-04 11:14:50
2024-05-04 14:14:40.926 [ 🐞 ] Scheduler is stopped 
2024-05-04 14:14:40.926 [ 🐞 ] Next (TEST-xxxx) event scheduled at 2024-05-04 11:14:41
2024-05-04 14:14:40.926 [ 🐞 ] Next (TEST-xxxx) event scheduled at 2024-05-04 11:14:41
2024-05-04 14:14:40.926 [ 🐞 ] Next (TEST) event scheduled at 2024-05-04 11:14:50
2024-05-04 14:14:40.926 [ 🐞 ] Scheduler is stopped 


In [418]:
bs.schedule_event('* * * * * */1', 'TEST-xxxx')

2024-05-04 14:10:14.397 [ 🐞 ] Next (TEST-xxxx) event scheduled at 2024-05-04 11:10:15


In [28]:
pd.Timedelta('1Min').asm8.item()

numpy.timedelta64(60000000000,'ns')

In [20]:
pd.Timedelta('4d').total_seconds()

345600.0

In [421]:
c.stop()

In [457]:
pd.Timedelta('0')

Timedelta('0 days 00:00:00')